In [90]:
import pandas as pd
import numpy as np
import hijri_converter


In [91]:
from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, errors = 'coerce')

df = pd.read_excel("RequiredData.xlsx", sheet_name="Reservation List Data", converters={'Reservation Check-out Date ':convert_date  , 'Cancellation Date  ':convert_date , 'Reservation Check-in Date':convert_date ,'Reservation Creation Date (MM-DD-YYYY)':convert_date})

df.dropna(inplace=True, subset=['Reservation Creation Date (MM-DD-YYYY)', 'Reservation Check-in Date', 'Reservation Check-out Date ', 'Cancellation Date  '])
df['Hijri Check-out Date'] = df['Reservation Check-out Date '].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df['Hijri Check-in Date'] = df['Reservation Check-in Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())


df.sort_values('Reservation Creation Date (MM-DD-YYYY)', inplace=True)

df['tmp'] = df['Reservation Creation Date (MM-DD-YYYY)'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df.insert(1, 'Reservation Creation Date Hijri', df.pop('tmp'))

df.reset_index(inplace=True, drop=True)
df.head(10)

,Reservation ID,Reservation Creation Date Hijri,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Reservation Total Price,Rooms,Hijri Check-out Date,Hijri Check-in Date
0,30,1441-02-20,2019-10-19,2020-01-11,2020-01-15,2019-10-26,1,4160.0160,2,1441-05-20,1441-05-16
1,49,1441-02-21,2019-10-20,2019-12-08,2019-12-14,2019-10-31,1,4439.9985,1,1441-04-17,1441-04-11
2,43,1441-02-21,2019-10-20,2019-12-30,2019-12-31,2019-12-30,1,910.0035,1,1441-05-05,1441-05-04
3,95,1441-02-25,2019-10-24,2020-01-11,2020-01-17,2019-10-30,1,32040.0360,5,1441-05-22,1441-05-16
4,197,1441-03-01,2019-10-29,2019-12-15,2019-12-19,2019-11-03,1,4960.0320,2,1441-04-22,1441-04-18
5,244,1441-03-02,2019-10-30,2019-11-01,2019-11-02,2019-10-31,3,684.9990,1,1441-03-05,1441-03-04
6,214,1441-03-02,2019-10-30,2019-11-01,2019-11-02,2019-10-31,1,609.9975,1,1441-03-05,1441-03-04
7,260,1441-03-03,2019-10-31,2019-12-25,2019-12-28,2019-10-31,1,2245.0050,1,1441-05-02,1441-04-28
8,305,1441-03-05,2019-11-02,2020-01-05,2020-01-07,2020-01-01,1,1640.0160,2,1441-05-12,1441-05-10
9,327,1441-03-06,2019-11-03,2019-12-31,2020-01-01,2019-11-03,1,3060.0150,4,1441-05-06,1441-05-05


In [92]:
from hijri_converter import Hijri, Gregorian
Hijri(1433, 9, 1)

Hijri(1433, 9, 1)

In [93]:
value_1 = df.loc[0,'Hijri Check-out Date']
value_2 = df.loc[5,'Hijri Check-out Date']


if value_1 > value_2:
    print("The first row is bigger.")
elif value_1 < value_2:
    print("The first row is smaller.")
else:
    print("The values are equal.")

The first row is bigger.


In [94]:
# Create a new DataFrame to store the filtered reservations
filtered_df = pd.DataFrame(columns=df.columns)


In [95]:
target_start_day_hijri = Hijri(1444,9,1) # Hijri date in the format 'yyyy-mm-dd'
target_end_day_hijri  = Hijri(1445,1,1) # Hijri date in the format 'yyyy-mm-dd'
is_different_year = (target_start_day_hijri.year!=target_end_day_hijri.year)

In [96]:
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg

def increment_hijri(hijri_date, dateOffset=1):
    dt_greg = hijri_to_greg(hijri_date)
    dt_greg += pd.DateOffset(days=dateOffset)
    incremented_hijri_date = Gregorian(dt_greg.year, dt_greg.month, dt_greg.day).to_hijri()
    return incremented_hijri_date

In [97]:
# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Extract the check-in and check-out  Hijri dates 
    check_in_date_hijri = row['Hijri Check-in Date']
    check_out_date_hijri = row['Hijri Check-out Date']
    target_start_day_hijri_temp = Hijri(check_in_date_hijri.year, target_start_day_hijri.month, target_start_day_hijri.day) 
    target_end_day_hijri_temp = Hijri(check_out_date_hijri.year + is_different_year, target_end_day_hijri.month, target_end_day_hijri.day)



# Create a list to store the rows for each day within the range
    rows = []

    # Iterate over each day within the range
    current_day = check_in_date_hijri
    while current_day <= check_out_date_hijri:
        # Check if the current day falls within the check-in and check-out dates
        if target_start_day_hijri_temp <= current_day <= target_end_day_hijri_temp:
            # Append the row with the current day to the list
            rows.append(list(row)+[current_day])

        current_day = increment_hijri(current_day)

    if (rows == []):
        continue

    # Create a DataFrame for the rows for this reservation
    reservation_df = pd.DataFrame(rows, columns=list(df.columns)+['Target Date'])

    # Concatenate the reservation DataFrame with the filtered DataFrame
    filtered_df = pd.concat([filtered_df, reservation_df], ignore_index=True)



In [98]:
filtered_df

,Reservation ID,Reservation Creation Date Hijri,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Reservation Total Price,Rooms,Hijri Check-out Date,Hijri Check-in Date,Target Date
0,2569,1441-04-06,2019-12-03,2020-04-30,2020-05-04,2020-02-05,1,0.0,20,1441-09-11,1441-09-07,1441-09-07
1,2569,1441-04-06,2019-12-03,2020-04-30,2020-05-04,2020-02-05,1,0.0,20,1441-09-11,1441-09-07,1441-09-08
2,2569,1441-04-06,2019-12-03,2020-04-30,2020-05-04,2020-02-05,1,0.0,20,1441-09-11,1441-09-07,1441-09-09
3,2569,1441-04-06,2019-12-03,2020-04-30,2020-05-04,2020-02-05,1,0.0,20,1441-09-11,1441-09-07,1441-09-10
4,2569,1441-04-06,2019-12-03,2020-04-30,2020-05-04,2020-02-05,1,0.0,20,1441-09-11,1441-09-07,1441-09-11
...,...,...,...,...,...,...,...,...,...,...,...,...
2446,16430,1442-11-09,2021-06-19,2021-06-24,2021-06-26,2021-06-23,1,2247.0,1,1442-11-16,1442-11-14,1442-11-16
2447,16435,1442-12-16,2021-07-26,2021-07-31,2021-08-03,2021-07-26,1,2545.2,1,1442-12-24,1442-12-21,1442-12-21
2448,16435,1442-12-16,2021-07-26,2021-07-31,2021-08-03,2021-07-26,1,2545.2,1,1442-12-24,1442-12-21,1442-12-22
2449,16435,1442-12-16,2021-07-26,2021-07-31,2021-08-03,2021-07-26,1,2545.2,1,1442-12-24,1442-12-21,1442-12-23


In [99]:
numeric_columns = [col for col in filtered_df.columns if 'date' not in col.lower()]
filtered_df[numeric_columns] = filtered_df[numeric_columns].apply(pd.to_numeric, errors='coerce').astype('float')
print(filtered_df.dtypes)

Reservation ID                                                                          float64
Reservation Creation Date Hijri                                                          object
Reservation Creation Date (MM-DD-YYYY)                                           datetime64[ns]
Reservation Check-in Date                                                        datetime64[ns]
Reservation Check-out Date                                                       datetime64[ns]
Cancellation Date                                                                datetime64[ns]
Type\nallotment (1),  sales allotment (2), direct hotel (3)or direct agent(4)           float64
Reservation Total Price                                                                 float64
Rooms                                                                                   float64
Hijri Check-out Date                                                                     object
Hijri Check-in Date                     

In [ ]:
def get_greg_cr_dates_per_target_hijri_year(df):
    

In [101]:
# draft: will not use this function currently
def get_cr_dates_per_year(df, are_dates_hijri=False):
    """
    Retrieve a list of DataFrame arrays containing reservation creation dates grouped by year.

    Parameters:
        df (pandas.DataFrame): The initial DataFrame.
        are_dates_hijri (bool): Flag indicating whether the dates are in Hijri format. 
                                Default is True.

    Returns:
        list: A list of DataFrame arrays, where each array contains reservation creation dates for a specific year.

    Example:
        >>> df = pd.DataFrame({'Reservation Creation Date (MM-DD-YYYY)': ['01-01-2022', '01-02-2022', '01-01-2023', '01-02-2023']})
        >>> cr_dates_per_year = get_cr_dates_per_year(df, False)
        >>> print(cr_dates_per_year)
        [['01-01-2022', '01-02-2022'], ['01-01-2023', '01-02-2023']]
    """

    df = df.copy()

    if are_dates_hijri:
        df['Hijri Year'] = df['Reservation Creation Date Hijri'].apply(lambda x: x.year)
        # Group the DataFrame by year
        grouped = df.groupby(df['Hijri Year'])

        # Create separate DataFrame arrays for each year
        cr_dates_per_year = [group['Reservation Creation Date Hijri'].tolist() for _, group in grouped]
    else:
        # Group the DataFrame by year
        grouped = df.groupby(df['Reservation Creation Date (MM-DD-YYYY)'].dt.year)

        # Create separate DataFrame arrays for each year
        cr_dates_per_year = [group['Reservation Creation Date (MM-DD-YYYY)'].tolist() for _, group in grouped]

    return cr_dates_per_year


In [102]:
def create_reservation_dataframe():
    """
    Create a dataframe with a column 'Reservation Creation Date' containing all the days in a specified year.

    Parameters:
        year (int): The year for which to create the dataframe.

    Returns:
        pandas.DataFrame: A dataframe with a single column 'Reservation Creation Date' containing all the days in the specified year.

    Example:
        >>> df = create_reservation_dataframe(2023)
        >>> print(df)
           Reservation Creation Date
        0                2023-01-01
        1                2023-01-02
        2                2023-01-03
        ...              ...
        364              2023-12-31
    """
    start_date = pd.to_datetime(f'{year}-01-01')
    end_date = pd.to_datetime(f'{year}-12-31')

    dates = pd.date_range(start=start_date, end=end_date, freq='D')

    df = pd.DataFrame({'Reservation Creation Date': dates})

    return df


def greg_to_hijri(greg_date):
    hijri_date = Gregorian(greg_date.year, greg_date.month, greg_date.day)
    hijri_date = hijri_date.to_hijri()
    return hijri_date

In [103]:
cr_dates_per_year = get_cr_dates_per_year(filtered_df)
target_dates_history_years = []
target_dates_history_dfs = []
for annually_creation_dates_greg in cr_dates_per_year:
    target_start_day_hijri_temp = Hijri(annually_creation_dates_greg[0].year, 
                                        target_start_day_hijri.month, 
                                        target_start_day_hijri.day)
    target_end_day_hijri_temp = Hijri(annually_creation_dates_greg[0].year + is_different_year, 
                                        target_end_day_hijri.month, 
                                        target_end_day_hijri.day)
    
    target_dates_range = []
    print('aaa')
    print(target_dates_range)
    current_day = target_start_day_hijri_temp
    while current_day <= target_end_day_hijri_temp:
        target_dates_range.append(current_day)
        current_day = increment_hijri(current_day)

    # Creating empty df
    annually_target_dates_history_df = create_reservation_dataframe(annually_creation_dates_greg[0].dt.year)
    # Creates columns where the column name is a Hijri() object (displayed as Y-m-d by __str()__ method in Hijri class)
    for column in target_dates_range:
        annually_target_dates_history_df[column] = np.nan
    
    for creation_date_greg in annually_creation_dates_greg:
        target_creation_dates_mask = (annually_target_dates_history_df['Reservation Creation Date'] == creation_date_greg)
        filtered_creation_dates_mask = (filtered_df['Reservation Creation Date (MM-DD-YYYY)'] == creation_date_greg)
        filtered_target_dates = [dt for dt in list(filtered_df[filtered_creation_dates_mask]['Target Date'])]
        filtered_rooms = list(filtered_df[filtered_creation_dates_mask]['Rooms'])
        
        target_date_to_rooms = {}
        for i, target_date in enumerate(filtered_target_dates):
            if target_date not in target_date_to_rooms.keys():
                target_date_to_rooms[target_date] = 0
            target_date_to_rooms[target_date] += filtered_rooms[i]    

        annually_target_dates_history_df.loc[target_creation_dates_mask, # to select row in ann_td_h_df where creation date is the current one in the loop
                                                list(target_date_to_rooms.keys()) # to update the column names of said row
                                            ] \
                                            = list(target_date_to_rooms.values()) # updates it to Rooms
         

    target_dates_history_years.append(annually_creation_dates_greg[0].year)
    target_dates_history_dfs.append(annually_target_dates_history_df)

target_dates_history_years

OverflowError: date out of range

In [104]:
target_dates_history_dfs[0]

IndexError: list index out of range

In [ ]:
target_dates_accum_history_dfs = target_dates_history_dfs.copy()
for tdacc_df in target_dates_accum_history_dfs:
    # cumulatively add each row to the row below it
    tdacc_df.fillna(0, inplace=True)
    tdacc_df[tdacc_df.columns[1:]] = tdacc_df[tdacc_df.columns[1:]].cumsum()

    # when reaching the row where "Reservation Creation Date" is equal to the Column date, 
    # fill that column's remaining rows (after that row) with 0s 
    for i, column_hijri in enumerate(tdacc_df.columns):
        if not isinstance(column_hijri, Hijri):
            continue 
        matching_dates_row_idx = tdacc_df.loc[tdacc_df['Reservation Creation Date']==hijri_to_greg(column_hijri)].index[0]
        if matching_dates_row_idx == len(tdacc_df)-1:
            continue
        tdacc_df.iloc[matching_dates_row_idx+1:, i] = np.nan

2020-04-24


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [ ]:
target_dates_accum_history_dfs[0]

,Reservation Creation Date,1444-09-01,1444-09-02,1444-09-03,1444-09-04,1444-09-05,1444-09-06,1444-09-07,1444-09-08,1444-09-09,...,1444-12-22,1444-12-23,1444-12-24,1444-12-25,1444-12-26,1444-12-27,1444-12-28,1444-12-29,1444-12-30,1445-01-01
0,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
weekly_target_dates_accum_dfs = []
for tdacc_df in target_dates_accum_history_dfs:
    new_columns = ['Reservation Creation Date']
    new_agg_values = [tdacc_df.iloc[:, 0].to_list()]
    df_columns = tdacc_df.columns
    for start_date_idx in range(1, len(df_columns), 7):
        # get column name of this week's range
        end_date_idx = min(start_date_idx+6, len(df_columns)-1)
        column_name = f'W{(start_date_idx//7)%4 + 1} - ' + df_columns[start_date_idx].month_name()
        if df_columns[start_date_idx].month != df_columns[end_date_idx].month:
            column_name += '/' + df_columns[end_date_idx].month_name()
        new_columns.append(column_name)

        # get sum of Rooms of this week's range as a single list
        new_agg_values.append(tdacc_df.iloc[:, start_date_idx:start_date_idx+7].sum(axis=1, skipna=True).to_list())

    weekly_tdacc_df = pd.DataFrame(np.array(new_agg_values).T, columns=new_columns)

    # convert all trailing zero values (per column) to np.NaN values
    for column in weekly_tdacc_df.columns:
        if 'date' in column.lower():
            continue
        mask = weekly_tdacc_df[column] != 0
        last_nonzero_index = mask[::-1].idxmax()
        if last_nonzero_index < len(weekly_tdacc_df)-1:
            weekly_tdacc_df.loc[last_nonzero_index + 1:, column] = np.nan

    weekly_target_dates_accum_dfs.append(weekly_tdacc_df)


In [ ]:
weekly_target_dates_accum_dfs[0]

,Reservation Creation Date,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Ramadhan/Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Shawwal/Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Qi’dah/Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah,W1 - Dhu al-Hijjah/Muharram
0,2023-01-01,0.0,0.0,0.0,0.0,0.0,30.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-01-02,0.0,0.0,0.0,0.0,0.0,30.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-01-03,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-01-04,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-01-05,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,2023-12-28,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362,2023-12-29,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,2023-12-30,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
